# Draft


Baking it in: Steps towards egraphs modulo theories.

What is "modulo theories"

SMT solvers - sat and egraph glue together diasprarte bits
EMT = SMT - SAT


eid + ctx as structured eid?


There are two tacts

- Generalize Enodes
- Generalize Union Find (which implies generalizing eids)

A simplier starting problem is Terms Modulo Theories

- canonizers
- Baking in unorderedness
- Hash Consing. Interned vs uninterned. The hash cons itself is a term bank

This is the same discussion as generalized enodes. With the caveat that smart constructors can deep match.


Do the theory subterms go in the term bank or don't they? If they do, are we saving anything?

Two other tacts:
- extract and assert
- SMT piggy back


You can turn a set of ground equations into an egraph.
And also extract them. Equations extraction. egraph(equations(e)) = e? If we don't included the term bank concept, no.
a = a can be used to enocde term bank, but I think it is conceptually interesting to make it seperate.
E + T = Egraph
extract_equations(egraph(eqs)) give a canonical form perhaps

Group union find is a generalized union find.
Idempotent theory




## Introduction
Simplification is the replacing of a complex thing with a better or simpler thing.

A rich framework to use to declare a simplification system is that of equational logic, where equivalences are specified as quantified equational axioms over terms.



These axioms and a signature implicitly define sets of equivalent terms. All the possible terms can be generated from the signature. Any two terms that match the left hand and right hand side of an equation.

Simplification is selecting a nicer term out of the set of equivalent ones.


A slightly different question is to start from one particular term and produce the set of equivalent terms. One can maintain a growing set of terms and increase it by applying the equational axioms to any term in the set at any subposition.


Egraphs are a data structure for compactly representing a set of ground equalities that supplies a fast check if equality between two terms are implied by the ground axioms. In a different mode, Egraphs can also be used to efficiently enumerate the set of equivalent terms for a given term. Via a dynamic programming approach, one can also extract a good equivalent term.

Ground terms by and large are easier to deal with and more well behaved that non-ground terms. Non ground terms can be seen as an optimization or abstract domain for finitely and effectively representing certain possibly infinite sets of terms, but this power does not come entirely for free.

For example, it is the case that for some equational systems, the equalities can be oriented into rewrite rules with good properties. When the rules are confluent and terminating, running then on a ground term will produce a unique normal form. In this manner, these rules can be used for equivalence checking of ground terms modulo this equational theory.

For non-ground terms, the situation is more complicated. A non-ground term represents in a sense a possibly infinite set of terms. It is not guaranteed that all of these terms are in the same equivalence class under the rules R. Therefore, there must be some kind of splitting possible. The only way for a rule to split the set represented by the term is to have a non trivial overlap with the non-ground function symbols of the term. This is called narrowing.

Q: Does narrowing terminate? Yes, maybe it does. The term ordering does not admit an infinite descending chain of even non ground terms. And it has substitiution property f(f(X)) -> X.   consider narrowing c(f(X), X). This cycles.  https://www.sciencedirect.com/science/article/pii/S0304397509005246? termination of narrowing revisisted



Another related approach to solving an equational system is Knuth bendix completion. Knuth Bendix completion when it succeeds produces a conlfuent and terminating rewrite systems out of an equational system. Knuth bendix completion of a ground equational system will succeed. The resulting ground rewrite system represents the set of ground equalities that produced it, but also gives a fast way to check if two ground terms are equal.



- Given two terms, are they equal?
- Given a term, generate all equal terms
- Given a term generate a best equal term




Egraph techniques from the get go have had a pragmatic bent. It is convenient from an implementation perspective to use ground terms. It is an easy to implement reasonable search heuristic to seek rewrite coming out of the cloud resulting from the initial term.
From this perspective, the completeness or incompleteness of some new variation of egraph rewriting compared to the stack version does not seem that interesting.


Some theories, important ones being associativity and commutativity, feel structural. They are difficult to orient, one direction of commutativty or assocation is arguably not "simpler" than the other, it being more of a marginal tie breaking case. These axioms are also related to commonly avaiable and studied data structures such as sets, multisets, and permutatations.





## Ground + A Does not work


An important no-go counterexample is the following:

String rewriting is in general a nasty thing. It is a sufficiently flexible framework to encode the execution of a turing machine (fairly directly by encoding the tape and the head). If one could somehow generate a terminating normalization procedure for any string equation system, then one could solve the halting problem by asking if the halt state and the initial state are equal. 

String rewriting can be encoded in term rewriting. A string can be represented as a binary concatenation on constants obeying associativity. The particular equations of a string rewrite system can be represented by ground equations. The only non ground rule needed is the associativity axiom.

A different encoding that in a sense bakes in a choice of the associativity (a more "list-like" representation) turns each character into a unary function symbol. This is sort of the fused combo of the character constants and concatenation  `a(X) = (a0 . X)`. To faithfully ecnode the string rewriting system, all the rules are now not ground, allowing an arbitrary suffix or prefix.



The ground Ground Knuth Bendix is terminating and will succeed. The reason is that the overlap relation between ground terms is the subterm relationship. A critical pair will be generated, but both terms in it will always be bounded from above by a term already existing in the system. There are only finitely many terms less than a given term in a particular well founded (... is this true? No. It is not. Maybe if I require it to be a substituion ordering?) What is produced is less than the original term in a total well founded order. Because the ordering is well founded this cannot go on forever. Because the order is total, all critical pairs produced can be oriented.

Regular Knuth bendix on non ground terms does not always succeed. Critical pairs can be generated which are not orientable because the pairs are unordered with respect to the necessarily partial term ordering on non ground terms. Ground terms orders can be total.

It is the totality of the term ordering which is crucial here.

Ground knuth bendix + the single associativity axiom will not always succeed. How does it fail?


Supposed we have a a set of equations which can be separated into a ground part G and and non ground part E. Using ordinary completion by assumtion let us say we can convert this system to G;R by merely choosing to ingore Knuth Bendix rules that manipulate G. Now we may do the same for G ,which will succeed.
Now the only critical pairs are between members of R and G.

When a non ground term (in R) makes a critical pair with one in G, it may do it by being a true subterm or by grounding one or more of the variables in an overlap.
This critical pair may not be fully grounded.

Ground Knuth Bendix + AC will succeed. Why is this the case? No it cannot. C cannot be oriented. But the system can be _ground completed_. A system is ground confluent if any ground term can be joined and ground terminating if .

An ordered rewriting system allows ordering constraints between the variables.
f(x,y) | f(x,y) > f(y,x) --> f(y,x)  # this is actually an infinite family of ground rules (?) But so was it always? No there's some kind of game being play about when the terms filling the variables get to be chosen. Static vs dynamic.
push inequalities down (we never hit variable rules. x,y are metavariables)
f(x,y) | x > y -> f(y,x).
These orderings may prune some critical pairs. We need to push constraints down to the variables by symbolically executing the term orderings.
Overlaps between these rules and ground rules are easy to prune.
```
f(x,y) > p(x,y) 
---------------   
```

AC is probably ground confluent by left associng and sorting. It is not so easy to see this directly from the definition.
ground + AC is less clear

We can split any rule without ordering constraints into multiple rules of the possible ordering constraints. This is because we assume the ground ordering is total.











For syntactic pattern matching of a single pattern against a term, anything except the top down approach feels ludicrous.






EIds as models

A term model is considered sometimes to be just syntax is disguise.




Baking in theories in term rewriting occurs in the form of E-unification or you can move off of the structural definitions of terms.

Unfailing completion is a theorem proving method. It is parametrized on a notion of E used for unification.
E-unficiation can be parametrized on a set of rewrite rules for the theory.



A different intution is 


# Unifying Enodes and Containers

A basic Enodes are an ordered record of eclass ids inputs arguments, a function symbol, and an output eclass id.

Extensions of the egraph have also allows Enodes to have arguments that are primitive datatypes like ground integers, strings, bools or rationals.

## Hash consing modulo theories
https://www.philipzucker.com/hashing-modulo/

A simpler case to consider is that of interning / hash consing terms. A useful features of hash consing is that equality of terms can be determined by a constant time pointer equality check.

To say that a binary symbol is commutative is to say that the order of the children does not matter. It is a simple matter to normalize a binary symbol of this kind by sorting it's arguments. This puts the term into a canonical form.

Given a confluent and terminating rewrite rule set, one can hash cons a term using smart constructors which check to see if the to be constructed term has rewrite rules applicable to it. As an optimization, these intermediate states do not necessarily have to be hash consed. There is an invariant of a hash consed tree that it is fully normalized with respect ot the rules. Placing a new function symbol at the head via a smart constructor may unlock some new rewrite rule at the head.

Sorting the arguments can be seen as applying ordered rewriting. The commutativity axiom cannot ever be ordered because it is too symmetric. One can wait until "runtime" to apply the rule using ordered rewriting.




For associative functions, it is very natural to allow multi-arity function symbols and flatten the arguments into a canonical list. When the smaert constructor peeks at the arguments, it can flatten.

A related but different approach is to maintain the binary function symbols but recursively do a list append.

The arguments of a function symbol can be

- multisets
- sets
- polynomials


- sorting
- flattening
- deduplicating
- polynomials

### Bottom up E-matching

Suppose we have some rules that are less well behaved we keep off separate from our built in ones.

We could take an equational theory and one by one add them to a Knuth Bendix procedure according to some heuristic. When or if the procedure fails, we can take the best subset of the equations that we can complete and leave the rest for later.
Or perhaps do Knuth Bendix, stop at some intermediate state `E;R` where E has all the critical pairs of R. We then treat R intrinsically, and treat E externally.

E-matching is finding a substitution such that the instantiated pattern is equal to a term modulo the equational theory. 

A top down strategy involves narrowing the pattern against the rules. The ground term is presumed fully normalized so the rules do not apply in the forward direction.

Q: What is the stopping condition?

A bottom up strategy involves expanding the ground term using the reversed rules. 


A matching problem starts with a pattern and a term supposed equal. This is a natural place to start top down matching.

In E-matching, there is not generic a priori guarantee that the variables will be bound to a subterm of the target. They may not even be bound to a term in an equivalence class of the subterms of the target. The terms must be generated by the rules and searched over somehow. This sucks.

Pattern matching is nice because the procedure is basically obvious. Pattern matching is not the most general thing we can conceive of. The pattern matching problem is an intermediary that is mathematically statable and efficiently implementable. If it was missing either of there characters, it would not be as prominent an idea.

Reasonable mechanisms can probably eventually be reconciled with a logical view. Case is point is negation as failure in logic programming. This is an extremely antural operational move in Prolog, but reconciling it with a logical perspective took a great many years of work.

Is the only logic worth talking about something close to classical first order logic? Maybe.


A different matching problems starts with a pattern and a bank of terms with which one might wish to bind the variables. 

I believe it is the case in general that one may encounter an infinite number of equivalence classes as subterms. This appears to be asking too much.

As an example, consider baking in linear expressions subject to a background theory of linear equalities. Anything on a line/hyperplane would also be a possible term.
We could perhaps cook up a way of describing 


Bottom up ematching is a particular strategy for relational ematching. 
In it's most brute force form, one scans over the databank of terms for every variable appearing in the pattern.
It can be optimized by pruning the possible bindings of variables by first doing a pass that finds an overapproximation of the sets the variables can bind to. This is akin to a WCOJ.


## What patterns are easily implementable




## Replacing Union Finds with Other Canonizers

The union find is a data structure for solving atomic equations. Given an atom, it can return a canonical atom for it's equivalence class.

There are other canonizer producing algorithms.

Atomic Knuth Bendix completion solves the same problem. The rewrite rules can be viewed as a manifestation of the union find. One difference is that Knuth Bendix is parametrized on a a term ordering. The union find can be done in a style with a determinstic ordering, or it can be done in a way that depends on the order of the asserted unions. This freedom enables the union find to keep itself flat and achieve a better asymptotic complexity.

Atomic multiset completion also  

Taking the row echelon form of a set of linear equations builds a canonizer.

Grobner bases form a canonizer for polynomial expressions under the assumption polynomial equalities.

You can also use an entire knuth bendix procedure as your canonizer producer. In this case it may not terminate though.


By enriching the union find to one of these other canonizers, we can push some of the work that would be done by the egraph or rules into the union find.





Some function symbols can be chosen to belong to enodes, and some to the structured eids. Baked in rules can only talk about the symbols in the strucutred eids and the other symbols are considered foreign.

At one extreme we have the egraph with it's atomic integer ids.

At the other extreme, the 


# Extract and Rewrite









https://www.youtube.com/watch?v=udORacqkExg&ab_channel=ACMSIGPLAN  WiP: Labeled Union-Find for Constraint Factorization


Completion without failure  https://www.cs.tau.ac.il/~nachum/papers/unfail-paper.pdf

Le Chenadec - Canonical Forms in Finitely Presented Algebras . Can't find it


normalized completion
https://drops.dagstuhl.de/storage/00lipics/lipics-vol021-rta2013/LIPIcs.RTA.2013.319/LIPIcs.RTA.2013.319.pdf
https://www.sciencedirect.com/science/article/pii/S0747717196900115  Normalized Rewriting: an Alternative to Rewriting modulo a Set of Equations

https://dl.acm.org/doi/pdf/10.1145/800017.800519 jouannaud kirchner - completion of a set of rules modlo a set of equations.

https://wiki.bordeaux.inria.fr/Helene-Kirchner/lib/exe/fetch.php?media=wiki:rsp.pdf  rewriting solving proving. Quite a table of contents.

t < s as an inference rule system


if finitariness is good, then actually strongly simplifying R is bad.


Build my own smt solver? My own outer Z3 solver.
Make a command line form that takes in smtlib (via z3's parser)

assoc has 3 variables.
A + C is ok though.
Bubble sort completion (a + (b + c)) -> (b (+ a + c))

You can N-theory it by statically choosing a depth N that rules have been theorified.

The ordering can never be stable.


Group union find. g x = g2 x. Overlap can be achieved by inverted the group element.
monoid union find is fine. Or string/semigroup action (no unit). This maps onto working always at the end of the string. And is ground equations basically. f(ff(endsymbols))


Completeness
herbrand https://www.youtube.com/watch?v=bdmigQsf_uY&ab_channel=LawrencePaulson
https://math.stackexchange.com/questions/2468277/intuition-about-herbrand-models

https://terrytao.wordpress.com/2009/04/10/the-completeness-and-compactness-theorems-of-first-order-logic/


Making complete refutation system out of egglog. You can collect all needed generations. Enough to simulate the needed
f(x) --> 


Set rewriting for idempotent. Set term orders? Set rewriting is very close or identities with ground ordered resolution. But set rewriting doesn't have removal replacement semantics. It has monotonic semantics.

Knuth bendix with constraints is an interesting paradigm. Overlap is normal overlap + combined contraints being satisfiable / consistent.

ground + C is maybe the simplest one. I basically ocnsidered the container version. But if we push C into the eid, we make eids C symbols  + opqaue constants. 

ground + I. everything is either opqaue a, or f(a). It might be better to rewrite _to_ `b -> f(a)` because absortbing is better than not.

Multiple I symbosls. f(g(f(a))). It is excessive to do this. As soon as we go to 

We can do top down ematching by delegating to a theory specific matching for the eid when the pattern hits symbols that belong to the eids.

You put everything into the eid that you'd need to identify

stratified egraphs - consider a structured eid who's theory is _also_ ground equations. The structured eid is now a term who only contains function symbols from lower egraphs or atomics eids. Canonization is by extraction from the lower theory.

G1 + G2 is obviosuly confluent and terminating even though a modular problem.


- relational AC-matching
- https://www.philipzucker.com/egraph2024_talk_done/
- https://www.philipzucker.com/linear_grobner_egraph/
- https://www.philipzucker.com/coegraph/
- https://www.philipzucker.com/string_knuth/
- https://www.philipzucker.com/smart_constructor_aegraph/
- https://www.philipzucker.com/multiset_rw/
- https://www.philipzucker.com/bottom_up/ normalizing containers


a theory is ground exendable if it remains completeable with ground equations.

Associativity is not groud extendable.


Enodes ~ syntax
eids ~ semantics

When you ask z3 for a model, it returns numerical semantic values.

Equality saturation starts as pure syntactic equality and gradually moves over closer to semantic equality (which is often never reaches).

There are two main approaches I've been trying (or that I'm aware of):

- Containers as smarter enodes
- structured / semantic eids

The enode structure I used is a function symbol + an ordered list of arguments. This is already kind of smart compared to a function symbol + a binary tuple of args. This tuple form can be interesting if you want to talk about partial application.


Destructive rewriting as a theory.
If you have a good rewrite theory over a system (terminating and confluent), you can already effectively quotient the syntax trees by that notion of equivalence be greedily applying the rules.

```
add(succ(X), Y) -> succ(add(X,Y))
add(Z, Y) -> Y
```


Combination problems
https://www.sciencedirect.com/science/article/pii/S0890540101931189 Deciding the Word Problem in the Union of Equational Theories

Canonization for disjoint unions of theories Sava Krstic  Sylvain Conchon

Disjoint signatures
Termination is not modular
Confluence is modular
Normalizing is modular

Ground rewrite rules / equalities are quite special.

The following no go counterexample crushes most hopes of generic good results.

Associativity + ground term equations = string equality and is undecidable.

You can build a normalizing rewrite rule system for associativity by associating to the right.
`(x . y) . z -> x . (y . z)`

Ok but actually string rewrite systems are rules of form (yadayda . X) -> (something . X). They are not ground. Yeah but associatvity can make them non ground
A ground _term_ equation corresponds more directly to the string rewrite that only applies at the end of the string. 

For disjoint signatures, there doesn't seem like their should be a problem. Why were eids integers in the first place? Who cares? The pieces of the other signature completely block rules from applying.


Pick a ground total termination ordering compatible with the rewrite relation.
COnsider the particular stratgey: orient the built in rules, All confluence pairs are now redundant by assumption.
Start orienting the ground equations. simplify reduces by the built ins. This is "putting into the egraph".
Any symbol not part of built ins can have a generated guy.




Ok, so we want non-disjoint signatures.


What terms are "in" the egraph:
1. A generative point of view (top down production of terms). Given an eid, we can build. The view is related to top down ematching
2. An acceptor point of view. If we insert of term bottom up into the egraph, does it change it or leave it the same? This view is related to bottom up ematching.

We can consider maybe adding new function symbols over 


Can we put eids and enodes on the same footing? If I had 3 theories I wanted to bake in. Does baked in go into eids? Or 2 notions of eids?

https://egraphs.zulipchat.com/#narrow/channel/328972-general/topic/Linear.20and.20Polynomial.20Equations/near/477290392


https://egraphs.zulipchat.com/#narrow/channel/424128-egg.2Fgeneral/topic/Translating.20between.20languages.20in.20an.20egraph/near/499111241 translation between mutiple languages slows stuff down.
Related to refinement egraph ideas?


|           |   UF    | Theory |
| ---       |  ---    |        |
| Sorts     | 
| Enodes    |
| Eclasses  |
| Functions |
| Rules     |   


Egraph modulo theories:
What is a theory?

https://pldi25.sigplan.org/home/egraphs-2025


In [2]:
class SortRef():
    def __init__(self, name):
        self.name = name
        self.uf = []
        # self.utree # to maintain eclasses
        # self.reason # to maintain provencnance / proof
    def find(self, i): # canonize
        while self.uf[i] != i:
            i = self.uf[i]
        return i
    def union(self, i, j):
        i = self.find(i)
        j = self.find(j)
        if i != j:
            self.uf[i] = j
    def make(self): # Const?
        self.uf.append(len(self.uf))
        return len(self.uf)-1
        

class FuncDeclRef(): # MergeDict
    def __init__(self, name, *sorts):
        self.data = {}
        self.sorts = sorts
    def __getitem__(self, args):
        assert len(args) == len(self.sorts) - 1
        return self.data[args]
    def __call__(self, *args):
        assert len(args) == len(self.sorts) - 1
        res = self.data.get(args)
        if res is None:
            res = self.default()
            self.data[args] = res
            return res
        else:
            return res
    def __setitem__(self, key, value):
        assert len(key) == len(self.sorts) - 1
        res = self.data.get(key)
        if res is None:
            self.data[key] = value
        else:
            self.data[key] = self.merge(res, value)
    def default(self):
        return self.sorts[-1].make()
    def merge(self,i,j):
        return self.sorts[-1].union(i,j)
    def rebuild(self):
        fnew = FuncDeclRef(self.name, *self.sorts)
        for k,v in self.data.items():
            newk = tuple(sort.canon(k) for k,sort in zip(k,self.sorts[:-1]))
            fnew[newk] = self.sorts[-1].canon(v)
        return fnew
 
Int = SortRef("Int")
add = FuncDeclRef("add", Int, Int, Int)
x = FuncDeclRef("x", Int)()
add(x,x)


1

In [7]:
class MergeDict(dict):
    def __setitem__(self, key, value):
        res = self.get(key)
        if res is None:
            super().__setitem__(key, value)
        else:
            super().__setitem__(key, self.merge(res, value))
    def merge(self, i, j):
        raise NotImplementedError
    
class MinDict(MergeDict):
    def merge(self, i, j):
        return min(i, j)

class MaxDict(MergeDict):
    def merge(self, i, j):
        return max(i, j)

d = MinDict()
d[1] = 2
d[1] = 3
assert d[1] == 2
d[1] = 1
assert d[1] == 1

In [ ]:
class MSFuncDeclRef(FuncDeclRef):
    # multiset funcdecl ref. 
    # Putting the "ACness" into the functions
    # Containers ~ enodes
    def rebuild(self):
        fnew = MSFuncDeclRef(self.name, *self.sorts)
        for k,v in self.data.items():
            fnew[k] = v
        return fnew


In [ ]:
class ACSort():
    # putting the "ACness" in the sort.
    # structured eids which are multisets
    # containers ~ eids
    def __init__(self, name, inner):
        self.name = name
        self.inner = inner
    def rebuild(self, xs : tuple): # rebuild / re-canonize
        return tuple(sorted([self.inner.rebuild(x) for x in xs]))

class ACISort():
    # ACI ~ sets
    def __init__(self, name, inner):
        self.name = name
        self.inner = inner
    def rebuild(self, xs : tuple): # rebuild / re-canonize
        

A notion of "kinds". Sorts may have different structured ids. The description or data of thse may lie in the Kind data strucctures.

In [ ]:
class UninterpretedKind():
    def __init__(self, name):
        self.uf = []
    def make(self):
        self.uf.append(len(self.uf))
        return len(self.uf)-1
    def union(self, i, j):
        return i
    def canon(self, i):
        return i
    def rebuild(self, i):
        return self.find(i)


class SortRef():
    def __init__(self, name, kind=None):
        self.name = name
        if kind is None:
            kind = UninterpretedKind()
        else:
            self.kind = kind
    

We could easily have multiple notions of equality.
They don't necessarily have to have much to do with each other. But we can't normalize across them except at the commonality.
There is a truly baked in equality that can be destructive.

We could only canonize when they agree they canonize. Or maintain a fixpoint of the canonization.


In [ ]:
class AtomicEq():
    def __init__(self, sort):
        self.sort = sort
        self.uf = []
    def canon(self, i):
        pass
    def rebuild():

class LinearEq():
    def __init__(self, sort):
        self.sort = sort
        self.uf = []
    def canon(self, i):
        pass
    def rebuild():
        pass # do gauss elim?

class RecursiveEq(): # FixEq
    pass # todo


class GroupActEq():
    pass # Can have group actions on the side of the UF

class StringEq(): ...

class FixEq(): ...

class UnionEq():
    def __init__():
        pass

We are quotienting by different equalities.
Having them play nice with each other is a theory combination problem.

Partitions ~ equaiovlanece relations.
Finest partition or corasest partition that contains the equivalence relations.
Equivalence relations form a lattice.
https://en.wikipedia.org/wiki/Equivalence_relation

We can bounce around two equivalence relations is we have a notion of well founded ordering we are following and get the canonizer for the combo (the coarser partition with more equalities).

I don't know how we'd get a canonizer for the intersection.

In [ ]:
class QuotientSort():
    def __init__(self, sort, eq):
        self.sort = sort
        self.eq = eq


In [ ]:
class ValueKind():
    # literals like literal integers
    def rebuild(self, i):
        return i
    
# This makes sense for Int where we have abstract Int expressions, but also sometimes literal int values. "Lifting" is somewhat awkward.
# Perhaps related to my notion of observation table for coegraphs. A ground value is a very powerful observation.


In [ ]:
class UnionKind(): # union or disjoint union / tagged union?
    def __init__(self, *kinds):
        self.kinds = kinds
    def rebuild(self, i):
        tag, inner = i
        return (tag, self.kinds[tag].rebuild(inner))
    def make(self, tag): # Hmm. Make takes stuff now...
        return (tag, self.kinds[tag].make())

class IntersectionKind():
    

From the ground rewrite system perspective, what are we doing?

A specialized notion of overlap
a compositional form of term ordering. Making a term ordering from a term ordering of the contained elements.
a notion of pattern matching and replacement / removal



TermKind

In [ ]:
def ACDict():
    

In [ ]:

class ExprRef():
    def __init__(self, id_, sort):
        self.sort = sort
        self.id_ = id_
    def __eq__(self, other):
        assert self.sort is other.sort
        self.sort.union(self, other)
    def eq(self, other):
        assert self.sort is other.sort
        return self.sort.find(self.id_) == self.sort.find(other.id_)

# herbrand

Why do the first order sentences have to be quantifier free to apply propositional logic theorems?

https://mathweb.ucsd.edu/~sbuss/ResearchWeb/herbrandtheorem/paper.pdf buss hebrand theorem

Graham says two forms of herbrand.
A direct proof translation form?



# eprover

Run eprover to saturation


In [9]:
from kdrag.all import *

T = smt.DeclareSort("T")
x,y,z = smt.Consts("x y z", T)
add = smt.Function("add", T, T, T)
kd.notation.add.register(T,add)

s = kd.solvers.EProverTHFSolver()
s.add(smt.ForAll([x,y,z], x + ( y + z) == (x + y) + z))
s.check()
print(s.res.stdout.decode())



# Preprocessing class: HSSSSMSSSSSNFFN.
# Scheduled 4 strats onto 8 cores with 300 seconds (2400 total)
# Starting new_ho_10 with 1500s (5) cores
# Starting ho_unfolding_6 with 300s (1) cores
# Starting sh4l with 300s (1) cores
# Starting ehoh_best_nonlift_rwall with 300s (1) cores
# new_ho_10 with pid 2024504 completed with status 9
# sh4l with pid 2024506 completed with status 9
# ehoh_best_nonlift_rwall with pid 2024507 completed with status 9
# ho_unfolding_6 with pid 2024505 completed with status 9
# Schedule exhausted
# SZS status GaveUp



In [30]:
%%file /tmp/ac.p

cnf(assoc, axiom, f(X,f(Y,Z)) = f(f(X,Y),Z)).
cnf(comm, axiom, f(X,Y) = f(Y,X)).
cnf(biz, axiom, f(a,f(b,c)) = f(c,f(a,b))).


Overwriting /tmp/ac.p


In [28]:
%%file /tmp/terms.p

f(c,f(q,r))
f(f(a,c),a)

Overwriting /tmp/terms.p


In [32]:
! eprover-ho /tmp/ac.p -S #| enormalizer -t /tmp/terms.p -f -


# Initializing proof state
# Scanning for AC axioms
# f is AC
# AC handling enabled
#
#cnf(i_0_5, plain, (f(X1,X2)=f(X2,X1))).
#
#cnf(i_0_4, plain, (f(f(X1,X2),X3)=f(X1,f(X2,X3)))).
#
#cnf(i_0_7, plain, (f(X1,f(X2,X3))=f(X3,f(X1,X2)))).
##
#cnf(i_0_10, plain, (f(X2,f(X1,X3))=f(X1,f(X2,X3)))).
######
#cnf(i_0_27, plain, (f(X3,f(X2,X1))=f(X1,f(X2,X3)))).
######################################################################################
# No proof found!
# SZS status Satisfiable
# Processed positive unit clauses:
cnf(i_0_4, plain, (f(f(X1,X2),X3)=f(X1,f(X2,X3)))).
cnf(i_0_5, plain, (f(X1,X2)=f(X2,X1))).
cnf(i_0_7, plain, (f(X1,f(X2,X3))=f(X3,f(X1,X2)))).
cnf(i_0_10, plain, (f(X1,f(X2,X3))=f(X2,f(X1,X3)))).
cnf(i_0_27, plain, (f(X1,f(X2,X3))=f(X3,f(X2,X1)))).

# Processed negative unit clauses:

# Processed non-unit clauses:

# Unprocessed positive unit clauses:

# Unprocessed negative unit clauses:

# Unprocessed non-unit clauses:




: